In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier, SGDRegressor, RidgeCV, LassoCV, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelEncoder, StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold
from sklearn.impute import KNNImputer, SimpleImputer
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score
from imblearn.over_sampling import SMOTEN


smoten_median_imputed_less_40 = pd.read_csv('smoten_median_imputed_less_40.csv', index_col=False)

pd.set_option("display.max_columns", None)


def ensembleScale_coef(scaleFunc1, estimatorFunc2, k_fold_int, x_array, y_array):
    k = k_fold_int
    kf = KFold(n_splits=k, shuffle=True, random_state=None)    
    clf = make_pipeline(scaleFunc1, estimatorFunc2)

    acc_score = []
    for train_index , test_index in kf.split(X):
        X_train , X_test = X.iloc[train_index,:],X.iloc[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]
    
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
     
    acc = accuracy_score(y_pred , y_test)
    acc_score.append(acc)
     
    avg_acc_score = sum(acc_score)/k
    print('accuracy of each fold - {}'.format(acc_score))
    print('Avg accuracy : {}'.format(avg_acc_score))

    outcome_labels = ['Intubation False', 'Intubation True']

    #print('parameters \n', clf.get_params())
    print('score \n', clf.score(X_train, y_train))
    print('feature_names_in_')
    print(clf.feature_names_in_)
    print()
    print('feature_importances_')
    print(clf[1].feature_importances_)
    print()
    for i in range(len(clf.feature_names_in_)):
        print(clf.feature_names_in_[i], ":" ,clf[1].feature_importances_[i])
    print()
    print('fit')
    print(clf.fit(X_train, y_train).n_features_in_, clf.fit(X_train, y_train).feature_names_in_)
    print()
    print('classification report \n', classification_report(y_test, y_pred, target_names=outcome_labels))
    micro_roc_auc_ovr = roc_auc_score(y_test, y_pred, multi_class="ovr", average="micro")
    print()
    print(f"Micro-averaged One-vs-Rest ROC AUC score:\n{micro_roc_auc_ovr:.2f}")
    print()
    print('\n')
    

print('Gradient Boost')

estimate = GradientBoostingClassifier(loss='log_loss', learning_rate=0.1, 
                                      n_estimators=100, subsample=1.0, 
                                      criterion='friedman_mse', min_samples_split=2, 
                                      min_samples_leaf=1, min_weight_fraction_leaf=0.0,
                                      max_depth=3, min_impurity_decrease=0.0, init=None, 
                                      random_state=None, max_features=None, verbose=0, 
                                      max_leaf_nodes=None, warm_start=False, 
                                      validation_fraction=0.1, n_iter_no_change=None, tol=0.0001, ccp_alpha=0.0)


print('### SMOTEN median impute ###')
print()

X = smoten_median_imputed_less_40.drop('outcome',axis= 1)
y = smoten_median_imputed_less_40['outcome']

print('StandardScaler')
print()
ensembleScale_coef(StandardScaler(), estimate, 5, X, y)

print('RobustScaler')
ensembleScale_coef(RobustScaler(), estimate, 5, X, y)


Gradient Boost
### SMOTEN median impute ###

StandardScaler

accuracy of each fold - [0.9744027303754266]
Avg accuracy : 0.19488054607508531
score 
 0.9779934584755404
feature_names_in_
['gender' 'age' 'heart_rate_min' 'heart_rate_max' 'heart_rate_mean'
 'mbp_min' 'mbp_max' 'mbp_mean' 'sbp_min' 'sbp_max' 'sbp_mean' 'dbp_min'
 'dbp_max' 'dbp_mean' 'temperature_min' 'temperature_max'
 'temperature_mean' 'glucose_min' 'glucose_max' 'wbc_min' 'wbc_max'
 'creatinine_min' 'creatinine_max' 'hemoglobin_min' 'hemoglobin_max'
 'pt_min' 'pt_max' 'urineoutput' 'sofa_coagulation' 'sofa_cardiovascular'
 'sofa_cns' 'sofa_renal' 'charlson_comorbidity_index']

feature_importances_
[3.84753833e-04 2.79831826e-01 3.28404246e-04 1.08763987e-04
 3.10119770e-02 8.08981434e-04 2.80395806e-04 7.26887901e-03
 7.75948990e-04 3.94503577e-04 3.38268674e-02 2.35190528e-03
 6.15283538e-04 1.10399181e-01 4.58342524e-05 8.63570351e-04
 4.36206508e-02 5.03996858e-04 7.44598182e-03 5.50114219e-03
 7.30962909e-03 3.4287